In [1]:
import pandas as pd
import numpy as np

In [2]:
path_name = 'C:/Users/Usuario/Downloads/DenunciesGUB-2023_GD_parcial_v2.csv'

In [3]:
delimiter = ';'

In [4]:
df_raw = pd.read_csv(path_name, 
                     encoding='latin-1', 
                     #keep_default_na=True,
                     #header = None,
                     #names = list(range(0,max_columns)),
                     #skiprows = 1,
                     delimiter=delimiter
                    )

C:\Users\Usuario\AppData\Local\Temp\ipykernel_8008\251316281.py:1: DtypeWarning: Columns (11,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(path_name,


In [5]:
# How many rows?
len(df_raw)

485834

In [6]:
for col in df_raw.columns:
    s = df_raw[col]
    vc = s.value_counts()
    print(f'Column: {col}')
    print('___')
    print(vc)
    print('___')

Column: Any Data Infraccio
___
2023    485834
Name: Any Data Infraccio, dtype: int64
___
Column: Expedient
___
1         1
323885    1
323897    1
323896    1
323895    1
         ..
161942    1
161941    1
161940    1
161939    1
485834    1
Name: Expedient, Length: 485834, dtype: int64
___
Column: Codi Infraccio
___
1211    126456
1071     94395
1660     43598
1072     23156
1210     21958
         ...  
1026         1
1581         1
1755         1
1145         1
3300         1
Name: Codi Infraccio, Length: 472, dtype: int64
___
Column: Codi infracció
___
SUPERAR LIMIT VELOCITAT SENYAL ESPECIFIC                        127469
MANCA DE COMPROVANT HORARI                                       94395
NO RESPECTAR RESTRICCIONS ZONES BAIXES EMISSIONS                 43598
SUPERAR LIMIT HORARI                                             23156
CIRCULA AMB EXCES DE VELOCITAT PER RAO DE VIA                    21958
                                                                 ...  
MARXA ENRE

In [7]:
df_raw.columns

Index(['Any Data Infraccio', 'Expedient', 'Codi Infraccio', 'Codi infracció',
       'Ordenança', 'Descripció ordenança', 'Precepte/Article infringit',
       'Carrer Infracció', 'Num carrer infracció', 'Barri Infracció',
       'Nom barri', 'Districte Infracció', 'Tipologia radar', 'Data Infraccio',
       'Hora Infraccio', 'Tipus de vehícle', 'Descripció tipus vehicle',
       'Marca del vehicle', 'Model del vehícle', 'Velocitat circulació',
       'Velocitat límit', 'Total Import', 'Imp cobrat'],
      dtype='object')

In [8]:
correct_character_dict = {'ó':'o','ç':'c','í':'i'}
def correct_string(string, correct_character_dict):
    corrected_string = [correct_character_dict[c] if c in correct_character_dict.keys() else c for c in string]
    return ''.join(corrected_string)
df_raw.columns = [correct_string(col,correct_character_dict) for col in df_raw.columns]

In [9]:
df_raw.head(2)[['Nom barri', 'Districte Infraccio', 'Tipologia radar']]

,Nom barri,Districte Infraccio,Tipologia radar
0,la Marina del Prat Vermell,3,Fix
1,la Marina del Prat Vermell,3,Fix


In [10]:
df_raw.columns

Index(['Any Data Infraccio', 'Expedient', 'Codi Infraccio', 'Codi infraccio',
       'Ordenanca', 'Descripcio ordenanca', 'Precepte/Article infringit',
       'Carrer Infraccio', 'Num carrer infraccio', 'Barri Infraccio',
       'Nom barri', 'Districte Infraccio', 'Tipologia radar', 'Data Infraccio',
       'Hora Infraccio', 'Tipus de vehicle', 'Descripcio tipus vehicle',
       'Marca del vehicle', 'Model del vehicle', 'Velocitat circulacio',
       'Velocitat limit', 'Total Import', 'Imp cobrat'],
      dtype='object')

In [11]:
# cols_to_keep = ['Codi infraccio','Descripcio ordenanca','Carrer Infraccio','Nom barri','Data Infraccio',
#                 'Hora Infraccio','Descripcio tipus vehicle','Marca del vehicle','Model del vehicle',
#                 'Velocitat circulacio','Velocitat limit','Total Import','Imp cobrat',
#                 'Tipologia radar','Expedient','Num carrer infraccio', 'Districte Infraccio'
#                ]

In [12]:
df = df_raw#[cols_to_keep].copy()

In [13]:
df.rename(columns={'Codi infraccio':'Descripcio infraccio'}, inplace=True)

In [14]:
df['Data Infraccio'] = pd.to_datetime(df['Data Infraccio'], format='%Y%m%d')

In [15]:
# I don't know what this stands for
# df['Hora Infraccio'] = df['Hora Infraccio'].apply(lambda x: x[0:2])

In [16]:
df['Hora Infraccio'].apply(lambda x: len(x)).value_counts()

6    371768
5    103737
4      8831
3      1498
Name: Hora Infraccio, dtype: int64

In [17]:
def correct_time(x):
    x =str(x).replace(' ','')
    if x=='':
        return np.nan
    else:
        return str(x).rjust(6,'0')[0:2]

In [18]:
df['Hora Infraccio'] = df['Hora Infraccio'].apply(lambda x: correct_time(x))

In [20]:
df['Hora Infraccio'].value_counts()

12    45363
10    40785
09    38820
16    38637
13    34101
17    33566
11    31532
15    31308
14    26912
19    25018
08    22551
18    21878
20    13597
07    11856
00    10329
21    10064
23     9777
22     8931
01     7787
05     5217
04     4731
02     4515
06     4308
03     3952
Name: Hora Infraccio, dtype: int64

In [21]:
df['Imp cobrat'].value_counts()

0         140866
0         106193
50         68719
25         32938
100        25633
           ...  
121,31         1
606,23         1
400            1
109,14         1
105            1
Name: Imp cobrat, Length: 179, dtype: int64

In [22]:
df['Imp cobrat'] = df['Imp cobrat'].apply(lambda x: float(str(x).replace(',','.')))

In [23]:
def treat_mixed_column_type(x):
    if isinstance(x,int):
        return x
    if isinstance(x,str):
        if x.replace(' ','')=='':
            return np.nan
    return int(x)

In [24]:
df['Districte Infraccio'] = df['Districte Infraccio'].apply(lambda x: treat_mixed_column_type(x))

In [25]:
# Remove trailing whitespaces from all str column types
def remove_whitespace(x):
    if isinstance(x,str):
        return x.strip()
    return x

In [26]:
for c in df.columns:
    if df[col].dtype!='int64':
        df[c] = df[c].apply(lambda x: remove_whitespace(x))

In [27]:
df.to_parquet('./denuncies_partial_2023.parquet')